In [1]:
from src.reader import Reader
from src.downloader import Downloader
from src.localreader import LocalReader, SHRINK, CENTERING

In [ ]:
#Download images Starting from 1st challange to 10th challange
#In other words dowload 10 1st subsets
#downloader = Downloader(start=0, end=10)
#downloader.download(path="data/AVA.txt")

In [2]:
reader = LocalReader(width=256, height=256, shaping=SHRINK)
for images in reader.read(start=0, end=2):
    print(images.shape)

[============================================================] 100.0% .531.tar.gz    
(234, 256, 256, 3)
[============================================================] 100.0% .430.tar.gz    
(828, 256, 256, 3)
